our task is to detect what kind of object is is

codes are:
octasome (h3h4)*2 + (h2ah2b)*2
tetrasome (h3h4)*2 + (h2ah2b)*0
hexasome (h3h4)*2 + (h2ah2b)*1
hemisome (h3h4)*1 + (h2ah2b)*1

how many such objects there is:
codes are
NLP 1,2,3,4... nucleosome-like pack

Which side is it:
for each NLP that whould be proximal and distal based on sequence

Current main idea:
find histone pairs by distance matrix between folds (use Alexeys code for metaltlections if can be adopted without direct creation of pynucl object)
find particles form distance matrices between histone pairs

detect side:
we need to make it stupid simple, side per se can be identified without DNA, than we have to guess by proximity somehow, i guess i will use anchocring arginines on h3 h4 dimer

In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
from pynucl.entity_detector import detect_entities,check_nucleosomes_sides
from pynucl.hist_features import hist_features
from pynucl.nucl_meta_select import create_elem_dict,nucl_sel_expand
from pynucl.seq_utils import seqrec_from_pdb,seq_from_mda,get_overhangL,get_overhangR
import MDAnalysis as mda
from pynucl.view_nucl import view_nucl
import io,requests
#u=mda.Universe('tmp/1kx5_ntm_opc.pdb')
u=mda.Universe('tmp/6hkt.pdb')
#u=mda.Universe('/home/_shared/_projects/nucl_gmx/h3-h4_2_tm_100DNA/h3-h4_tm_DNA.pdb')
#u=mda.Universe('/home/_shared/_projects/nucl_gmx/h3-h4_2_tm_80DNA/tetr80_relaxed.pdb')
#u=mda.Universe('tmp/5gse.pdb')
#u=mda.Universe('/home/_shared/_projects/nucl_gmx/')
#h=io.StringIO(requests.get('https://files.rcsb.org/download/%s.pdb'%('6VEN')).content.decode("utf-8") )
#u=mda.Universe(h,format='PDB')

In [51]:
%%time
in_dict=detect_entities(u)
import numpy as np

CPU times: user 3.77 s, sys: 1.45 s, total: 5.22 s
Wall time: 15 s


In [52]:

components=in_dict
seqs={}
seqs={k:{} for k in  components.keys()}#populate with keys


for k in  components.keys():
    if( components[k]['entity']=='histone'):
        seqs[k]['strseq']= components[k]['sequence']


##Step 2.3.
# Let's calculate overhang of fullseq over strseq and put it to  seqs[k]['overhang']
# Let's put the first residue id too.
for k in  components.keys():
    if( components[k]['entity']=='histone'):
        seqs[k]['overhangL']=seqs[k]['overhangR']=0
        resids= u.select_atoms('segid %s and (protein)'%k).residues.resids
        seqs[k]['fullseq']=seqs[k]['strseq'][0]+''.join(['X'*(diff-1) + seqs[k]['strseq'][i+1] for i,diff in enumerate(np.diff(resids))])
        seqs[k]['resid_start']= resids[0]


##Step 2.5. Get sequence features
# We supply every chain where possible with a list of SeqFeature Objects.
# These follow the numbering of fullseq
# 
seq_features={}
for k in components.keys():
    if( components[k]['entity']=='histone'):
        seq_features[k]=hist_features(seqs[k]['fullseq'])
elements_dict=create_elem_dict(components,seqs,seq_features)

In [53]:
%%time
check_nucleosomes_sides(components,elements_dict,u,seq_features)

CPU times: user 4.51 s, sys: 5.06 ms, total: 4.51 s
Wall time: 4.54 s


{'0': {'entity': 'histone',
  'type': 'H2A',
  'variant': 'canonical_H2A',
  'organism': 'Mus',
  'sequence': 'AKTRSSRAGLQFPVGRVHRLLRKGNYSERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGRVTIAQGGVLPNIQAVLLPK',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 2,
  'nlp_type': 'octamer'},
 '1': {'entity': 'histone',
  'type': 'H2B',
  'variant': 'canonical_H2B',
  'organism': 'Loxodonta',
  'sequence': 'KRSRKESYSIYVYKVLKQVHPDTGISSKAMGIMNSFVNDIFERIAGEASRLAHYNKRSTITSREIQTAVRLLLPGELAKHAVSEGTKAVTKYTSA',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 2,
  'nlp_type': 'octamer'},
 '2': {'entity': 'histone',
  'type': 'H2A',
  'variant': 'canonical_H2A',
  'organism': 'Mus',
  'sequence': 'TRSSRAGLQFPVGRVHRLLRKGNYSERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGRVTIAQGGVLPNIQAVLLPK',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 5,
  'nlp_type': 'oct

In [196]:
view=view_nucl(u,color='dull',onlyNcp=False)
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H3 and alpha3',elements_dict),color='red',name='H3 and alpha3')
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and (( H4 or H2B ) and alpha3 )',elements_dict),color='blue',name='(H4 or H2B) and alpha3')
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and (( H4 or H2A ) and beta3 )',elements_dict),color='silver',name='(H4 or H2A) and beta3')
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H2A and loopL1',elements_dict),color='#FF00FF',name='H2A and loopL1')
view.set_size(800,600)
view


NGLWidget()

In [90]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist,squareform,cdist
import seaborn as sns
from itertools import combinations,product,chain
from pymolint import mol_int
import copy

In [159]:
'segid B' in elements_dict['hist_folds']

True

In [153]:
# get hist pairs
coms={}
hist_folds=u.select_atoms(nucl_sel_expand('hist_folds',elements_dict))
for k in components.keys():
    if( components[k]['entity']=='histone') and (len(seq_features[k])>1):
        coms[k]=hist_folds.select_atoms(f'name CA and segid {k}').center_of_geometry()
        
com_list=[item for key,item in coms.items()]
segid_list=[key for key,item in coms.items()]
pairthreshold=10 #A
H3H4_pairs=[]
H2AH2B_pairs=[]
other_pairs=[]
for pair,distance in zip(combinations(segid_list,2),pdist(com_list)):
    if distance<=pairthreshold:
        hist1,hist2=components[pair[0]]['type'],components[pair[1]]['type']
        if (set((hist1,hist2)).issubset(['H3','H4']) and (hist1!=hist2)):      
            H3H4_pairs.append(pair)
        elif (set((hist1,hist2)).issubset(['H2A','H2B']) and (hist1!=hist2)):
            H2AH2B_pairs.append(pair)
        else:
            other_pairs.append(pair)
H3H4_pairs

[('A', 'B'), ('E', 'F')]

In [154]:
# find tetrameres
coms={}
alpha3=u.select_atoms(nucl_sel_expand('H3 and alpha3',elements_dict))
for pair in H3H4_pairs:
    coms[pair]=alpha3.select_atoms(f'name CA and segid {" ".join(pair)}').center_of_geometry()
com_list=[item for key,item in coms.items()]
pair_list=[key for key,item in coms.items()]

H3H3threshold=15 #A
H3H4_H3H4_list=[]
for pair,distance in zip(combinations(pair_list,2),pdist(com_list)):
    if distance<=H3H3threshold:
        H3H4_H3H4_list.append(list(pair))
H3H4_H3H4_list

[[('A', 'B'), ('E', 'F')]]

In [151]:
#find H2AH2B_H2AH2B tetrameres
coms={}
loopL1=u.select_atoms(nucl_sel_expand('H2A and loopL1',elements_dict))
for pair in H2AH2B_pairs:
    coms[pair]=loopL1.select_atoms(f'name CA and segid {" ".join(pair)}').center_of_geometry()

    com_list=[item for key,item in coms.items()]
pair_list=[key for key,item in coms.items()]

H2AH2Bthreshold=20 #A
H2AH2B_H2AH2B_list=[]
for pair,distance in zip(combinations(pair_list,2),pdist(com_list)):
    if distance<=H2AH2Bthreshold:
        H2AH2B_H2AH2B_list.append(list(pair))
H2AH2B_H2AH2B_list

[[('C', 'D'), ('G', 'H')]]

In [95]:
# find hemisomes
H3H4_H2AH2B_same_side_list=[]
H3H4_H2AH2B_opposed_side_list=[]
H3H4_H2AH2B_outlier_list=[]
beta3_H4=u.select_atoms(nucl_sel_expand('name CA and beta3 and H4',elements_dict))
beta3_H2A=u.select_atoms(nucl_sel_expand('name CA and beta3 and H2A',elements_dict))

alpha3_H4=u.select_atoms(nucl_sel_expand('name CA and alpha3 and H4',elements_dict))
alpha3_H2B=u.select_atoms(nucl_sel_expand('name CA and alpha3 and H2B',elements_dict))

opposed_side_threshold=10
same_side_threshold=20
for h3h4,h2ah2b in product(H3H4_pairs,H2AH2B_pairs):
    #opposed (H4 or H2A) and beta3
    beta3_H4_com = beta3_H4.select_atoms(f'segid {" ".join(  h3h4)}').center_of_geometry()
    beta3_H2A_com=beta3_H2A.select_atoms(f'segid {" ".join(h2ah2b)}').center_of_geometry()
    opposed_side_dist=np.linalg.norm(beta3_H4_com-beta3_H2A_com)
    
    #same (H4 or H2B) and alpha3
    alpha3_H4_com = alpha3_H4.select_atoms(f'segid {" ".join(  h3h4)}').center_of_geometry()
    alpha3_H2B_com=alpha3_H2B.select_atoms(f'segid {" ".join(h2ah2b)}').center_of_geometry()
    
    same_sede_dist=np.linalg.norm(alpha3_H4_com-alpha3_H2B_com)
    #same_sede_dist==0 is needed as selection can be empty
    if (opposed_side_dist<=opposed_side_threshold) and ((same_sede_dist >= same_side_threshold) or(same_sede_dist==0) ):
        H3H4_H2AH2B_opposed_side_list.append([h3h4,h2ah2b])
    elif ((opposed_side_dist>=opposed_side_threshold) or (opposed_side_dist==0)) and (same_sede_dist <= same_side_threshold):
        H3H4_H2AH2B_same_side_list.append([h3h4,h2ah2b])
    elif (opposed_side_dist<=opposed_side_threshold) and (same_sede_dist <= same_side_threshold):
        H3H4_H2AH2B_outlier_list.append([h3h4,h2ah2b])
if len(H3H4_H2AH2B_outlier_list)!=0:
    print('Warning, detected strange geometry during hemisome detection')
print(H3H4_H2AH2B_opposed_side_list)
print(H3H4_H2AH2B_same_side_list)
#print(H3H4_H2AH2B_outlier_list)

[[('A', 'B'), ('G', 'H')]]
[]


In [100]:
#we will have to order it after
nlp_list=[]
#1. add all tetrameres
nlp_list.extend(copy.deepcopy(H3H4_H3H4_list))
#2. add loose h3h4 dimers
for H3H4_pair in H3H4_pairs:
    if not(H3H4_pair in chain.from_iterable(nlp_list)):
        nlp_list.append([H3H4_pair]) 
#3. add same side dimers

for H3H4_H2AH2B in H3H4_H2AH2B_same_side_list:
    for nlp in nlp_list:
        if (H3H4_H2AH2B[0] in nlp):   
            nlp.append(H3H4_H2AH2B[1])
        elif (H3H4_H2AH2B[1] in nlp):
            nlp.append(H3H4_H2AH2B[0])
            
# #4. check that opposed dimers are in nlps
for H3H4_H2AH2B in H3H4_H2AH2B_opposed_side_list:
    if not set(H3H4_H2AH2B).issubset(chain.from_iterable(nlp_list)):
        print('Loose H3H4 H2AH2B tetramer detected segments:',H3H4_H2AH2B)
        for nlp in nlp_list:
            if (H3H4_H2AH2B[0] in nlp):   
                nlp.append(H3H4_H2AH2B[1])
            elif (H3H4_H2AH2B[1] in nlp):
                nlp.append(H3H4_H2AH2B[0])
# #5. check that h2a h2b dimers are in nlps
for H2AH2B_H2AH2B in H2AH2B_H2AH2B_list:
    if not set(H2AH2B_H2AH2B).issubset(chain.from_iterable(nlp_list)):
        print('Loose H3H4 H2AH2B tetramer detected segments:',H2AH2B_H2AH2B)

# #6. create dictionary of nlps with anotations
# nlp_list
nlp_list

Loose H3H4 H2AH2B tetramer detected segments: [('A', 'B'), ('G', 'H')]


[[('A', 'B'), ('G', 'H')]]

In [121]:
nlp_dict={}
for i,nlp in enumerate(nlp_list):
    indict={}
    indict['all']=list(chain.from_iterable(nlp))
    indict['all'].sort()
    indict['H3']=[segid for segid in indict['all'] if components[segid]['type']=='H3']
    indict['H4']=[segid for segid in indict['all'] if components[segid]['type']=='H4']
    indict['H2A']=[segid for segid in indict['all'] if components[segid]['type']=='H2A']
    indict['H2B']=[segid for segid in indict['all'] if components[segid]['type']=='H2B']
    indict['H3_H4']=[dimer for dimer in nlp if dimer in H3H4_pairs]
    indict['H2A_H2B']=[dimer for dimer in nlp if dimer in H2AH2B_pairs]
    indict['sides']=[[dimer] for dimer in indict['H3_H4']]
    for dimer in indict['sides']:
        for same_side_tetramer in H3H4_H2AH2B_same_side_list:
            if dimer[0] in same_side_tetramer:
                dimer.extend([other_dimer for other_dimer in same_side_tetramer if other_dimer not in dimer])
                
    if len(indict['sides'])==1 and len(H3H4_H2AH2B_opposed_side_list)==1:
        if indict['sides'][0][0] in H3H4_H2AH2B_opposed_side_list[0]:
            indict['sides'].append([other_dimer for other_dimer in opposed_side_tetramer if other_dimer not in dimer])
        
    if len(indict['sides'])==2:
        if len(indict['H2A_H2B'])==2:
            indict['type']='octamer'
        elif len(indict['H2A_H2B'])==1:
            if len(indict['H3_H4'])==2:
                indict['type']='hexamer'
            elif len(indict['H3_H4'])==1:
                indict['type']='split_octasome'
        elif len(indict['H2A_H2B'])==0:
            indict['type']='H3_H4_tetramer'
    elif len(indict['sides'])==1:
        if len(indict['sides'][0])==2:
            indict['type']='hemisome'
        elif len(indict['H3_H4'])==1:
            indict['type']='H3_H4_dimer'
        elif len(indict['H2A_H2B'])==1:
            indict['type']='H2A_H2B_dimer'
    nlp_dict[i]=indict
nlp_dict   

{0: {'all': ['A', 'B', 'G', 'H'],
  'H3': ['A'],
  'H4': ['B'],
  'H2A': ['G'],
  'H2B': ['H'],
  'H3_H4': [('A', 'B')],
  'H2A_H2B': [('G', 'H')],
  'sides': [[('A', 'B')], [('G', 'H')]],
  'type': 'split_octasome'}}

In [62]:
# now we have to deal with orientation
view=view_nucl(u,color='dull',onlyNcp=True)
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H3 and loopL1',elements_dict),color='red',name='loopL1')
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H3 and loopL2',elements_dict),color='blue',name='loopL2')
view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H3 and mgarg1 ',elements_dict),color='silver',name='loopL3')
#view.add_mda_representaton('spacefill',nucl_sel_expand('name CA and H2A and loopL1',elements_dict),color='#FF00FF',name='H2A and loopL1')
view.set_size(800,600)
view

NGLWidget()

In [129]:
top_dna_segid= [segid for segid,component in components.items() if (component['entity']=='DNA') and (component['type']=='DNAtop' )][0]
#print(top_dna_segid)
for nlpid,nlp in nlp_dict.items():
    nlp['sideids']=[]
    for sideid,side_sigids in enumerate(nlp['sides'],1):
        side_segid_list=list(chain.from_iterable(side_sigids))
        h3_one_sel=u.select_atoms(f'segid {" ".join(side_segid_list)}')

        point1=h3_one_sel.select_atoms(nucl_sel_expand('name CA and H3 and loopL1',elements_dict)).center_of_geometry()
        point2=h3_one_sel.select_atoms(nucl_sel_expand('name CA and H3 and mgarg1',elements_dict)).center_of_geometry()
        point3=h3_one_sel.select_atoms(nucl_sel_expand('name CA and H3 and loopL2',elements_dict)).center_of_geometry()


        top_chanin_sel=u.select_atoms(f'segid {top_dna_segid} and name P')
        top_chanin_sel_pos=top_chanin_sel.positions
        top_chanin_sel_resids=top_chanin_sel.resids
        points=[point for point in [point1,point2,point3] if len(point)==3]
        if len(points)==2:
            print('Warning either some histone features missing or mapping is wrong')
        if len(points)>=2:
            dist_mat=cdist(points,top_chanin_sel_pos)
            key_rid=top_chanin_sel_resids[np.argmin(dist_mat,axis=1)]
            if np.all(np.diff(key_rid)>=0):
                nlp['sideids'].append(1)
            else:
                nlp['sideids'].append(2)
        else:
            if len(side_segid_list)==2 and set([components[segid]['type'] for segid in side_segid_list]).issubset(['H2A','H2B']):
                nlp['sideids'].append(1 if nlp['sideids'][-1]==2 else 2)
                print('guessed side for chains '+" ".join(side_segid_list))
            else:
                print('cant detect side for chains '+" ".join(side_segid_list))

# #6. create dictionary of nlps with anotations
# nlp_list
nlp_list
for nlpid, nlp in nlp_dict.items():
    for side_sigids,sideid in zip(nlp['sides'],nlp['sideids']):
        for segid in chain.from_iterable(side_sigids):
            component=components[segid]
            component['side']=sideid
            component['nlp_id']=nlpid
            component['nlp_type']=nlp['type']
components

guessed side for chains G H


{'A': {'entity': 'histone',
  'type': 'H3',
  'variant': 'canonical_H3',
  'organism': 'Gallus',
  'sequence': 'KPHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLRFQSSAVMALQEASEAYLVALFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 1,
  'nlp_id': 0,
  'nlp_type': 'split_octasome'},
 'B': {'entity': 'histone',
  'type': 'H4',
  'variant': 'canonical_H4',
  'organism': 'Xenopus',
  'sequence': 'DNIQGITKPAIRRLARRGGVKRISGLIYEETRGVLKVFLENVIRDAVTYTEHAKRKTVTAMDVVYALKRQGRTLYGFG',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 1,
  'nlp_id': 0,
  'nlp_type': 'split_octasome'},
 'G': {'entity': 'histone',
  'type': 'H2A',
  'variant': 'canonical_H2A',
  'organism': 'Xenopus',
  'sequence': 'KTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAVRNDEELNKLLGRVTIAQGGVLPNIQSVLLPKK',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 0,
  'nlp_type': 'split_oct

In [291]:
nlp_dict

{0: {'all': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
  'H3': ['A', 'E'],
  'H4': ['B', 'F'],
  'H2A': ['C', 'G'],
  'H2B': ['D', 'H'],
  'H3_H4': [('A', 'B'), ('E', 'F')],
  'H2A_H2B': [('C', 'D'), ('G', 'H')],
  'sides': [[('A', 'B'), ('C', 'D')], [('E', 'F'), ('G', 'H')]],
  'type': 'octamer',
  'sideids': [1, 2]},
 1: {'all': ['K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R'],
  'H3': ['K', 'O'],
  'H4': ['L', 'P'],
  'H2A': ['M', 'Q'],
  'H2B': ['N', 'R'],
  'H3_H4': [('K', 'L'), ('O', 'P')],
  'H2A_H2B': [('M', 'N'), ('Q', 'R')],
  'sides': [[('K', 'L'), ('M', 'N')], [('O', 'P'), ('Q', 'R')]],
  'type': 'octamer',
  'sideids': [1, 2]},
 2: {'all': ['0', '1', 'U', 'V', 'W', 'X', 'Y', 'Z'],
  'H3': ['U', 'Y'],
  'H4': ['V', 'Z'],
  'H2A': ['0', 'W'],
  'H2B': ['1', 'X'],
  'H3_H4': [('U', 'V'), ('Y', 'Z')],
  'H2A_H2B': [('W', 'X'), ('0', '1')],
  'sides': [[('U', 'V'), ('W', 'X')], [('Y', 'Z'), ('0', '1')]],
  'type': 'octamer',
  'sideids': [1, 2]},
 3: {'all': ['a', 'b', 'c', 'd', 'e', 'f

In [292]:
for nlpid, nlp in nlp_dict.items():
    for side_sigids,sideid in zip(nlp['sides'],nlp['sideids']):
        for segid in chain.from_iterable(side_sigids):
            component=components[segid]
            component['side']=sideid
            component['nlp_id']=nlpid
            component['nlp_type']=nlp['type']
            
                                  
        
    

In [293]:
components

{'0': {'entity': 'histone',
  'type': 'H2A',
  'variant': 'canonical_H2A',
  'organism': 'Mus',
  'sequence': 'AKTRSSRAGLQFPVGRVHRLLRKGNYSERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGRVTIAQGGVLPNIQAVLLPK',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 2,
  'nlp_type': 'octamer'},
 '1': {'entity': 'histone',
  'type': 'H2B',
  'variant': 'canonical_H2B',
  'organism': 'Loxodonta',
  'sequence': 'KRSRKESYSIYVYKVLKQVHPDTGISSKAMGIMNSFVNDIFERIAGEASRLAHYNKRSTITSREIQTAVRLLLPGELAKHAVSEGTKAVTKYTSA',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 2,
  'nlp_type': 'octamer'},
 '2': {'entity': 'histone',
  'type': 'H2A',
  'variant': 'canonical_H2A',
  'organism': 'Mus',
  'sequence': 'TRSSRAGLQFPVGRVHRLLRKGNYSERVGAGAPVYLAAVLEYLTAEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGRVTIAQGGVLPNIQAVLLPK',
  'has_water': False,
  'has_ions': False,
  'has_other': False,
  'side': 2,
  'nlp_id': 5,
  'nlp_type': 'oct